# Deep Nets with TF Abstractions


# The Data

In [1]:
from sklearn.datasets import load_wine

In [2]:
wine_data = load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [5]:
print(wine_data.DESCR)

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [5]:
feat_data = wine_data['data']

In [6]:
labels = wine_data['target']

### Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaled_x_train = scaler.fit_transform(X_train)

In [12]:
scaled_x_test = scaler.transform(X_test)

# Abstractions

## Estimator API

In [13]:
import tensorflow as tf

In [14]:
from tensorflow import estimator 

In [15]:
X_train.shape

(124, 13)

In [16]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[13])]

In [17]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\satya\\AppData\\Local\\Temp\\tmpd18re4m8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001268E3D6E08>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [19]:
deep_model.train(input_fn=input_fn,steps=500)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\satya\AppData\Local\Temp\tmpd18re4m8\model.ckpt.
INFO:tensorflow:loss = 11.321894, 

In [20]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [21]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\satya\AppData\Local\Temp\tmpd18re4m8\model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [22]:
predictions = [p['class_ids'][0] for p in preds]

In [23]:
from sklearn.metrics import confusion_matrix,classification_report

In [24]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.83      0.53      0.65        19
           1       0.67      0.82      0.73        22
           2       0.87      1.00      0.93        13

    accuracy                           0.76        54
   macro avg       0.79      0.78      0.77        54
weighted avg       0.77      0.76      0.75        54



____________
______________

# TensorFlow Keras

### Create the Model

In [25]:
from tensorflow.contrib.keras import models

In [26]:
dnn_keras_model = models.Sequential()

### Add Layers to the model

In [27]:
from tensorflow.contrib.keras import layers

In [28]:
dnn_keras_model.add(layers.Dense(units=13,input_dim=13,activation='relu'))

In [29]:
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))

In [30]:
dnn_keras_model.add(layers.Dense(units=3,activation='softmax'))

### Compile the Model

In [31]:
from tensorflow.contrib.keras import losses,optimizers,metrics

# explore these
### losses.

### optimizers.

In [34]:
losses.sparse_categorical_crossentropy

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)>

In [35]:
dnn_keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train Model

In [36]:
dnn_keras_model.fit(scaled_x_train,y_train,epochs=50)

Train on 124 samples
Epoch 1/50
124/124 [==============================] - 1s 6ms/sample - loss: 1.1251 - acc: 0.2581
Epoch 2/50
124/124 [==============================] - 0s 185us/sample - loss: 1.1157 - acc: 0.3306
Epoch 3/50
124/124 [==============================] - 0s 198us/sample - loss: 1.1067 - acc: 0.3548
Epoch 4/50
124/124 [==============================] - 0s 145us/sample - loss: 1.0974 - acc: 0.4113
Epoch 5/50
124/124 [==============================] - 0s 145us/sample - loss: 1.0873 - acc: 0.4597
Epoch 6/50
124/124 [==============================] - 0s 185us/sample - loss: 1.0771 - acc: 0.5000
Epoch 7/50
124/124 [==============================] - 0s 145us/sample - loss: 1.0672 - acc: 0.5161
Epoch 8/50
124/124 [==============================] - 0s 210us/sample - loss: 1.0563 - acc: 0.5403
Epoch 9/50
124/124 [==============================] - 0s 137us/sample - loss: 1.0448 - acc: 0.5403
Epoch 10/50
124/124 [==============================] - 0s 206us/sample - loss: 1.0325 - ac

In [37]:
predictions = dnn_keras_model.predict_classes(scaled_x_test)

In [38]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88        24
           1       0.68      1.00      0.81        15
           2       1.00      0.87      0.93        15

    accuracy                           0.87        54
   macro avg       0.89      0.89      0.87        54
weighted avg       0.91      0.87      0.88        54



# Layers API

https://www.tensorflow.org/tutorials/layers

## Formating Data

In [63]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [64]:
wine_data = load_wine()
feat_data = wine_data['data']
labels = wine_data['target']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

In [66]:
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

### ONE HOT ENCODED

In [68]:
onehot_y_train = pd.get_dummies(y_train)
one_hot_y_test = pd.get_dummies(y_test)

### Parameters

In [69]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3
learning_rate = 0.01

In [70]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

### Placeholder

In [71]:
X = tf.placeholder(tf.float32,shape=[None,num_feat])
y_true = tf.placeholder(tf.float32,shape=[None,3])

### Activation Function

In [72]:
actf = tf.nn.relu

### Create Layers

In [73]:
hidden1 = fully_connected(X,num_hidden1,activation_fn=actf)

In [74]:
hidden2 = fully_connected(hidden1,num_hidden2,activation_fn=actf)

In [75]:
output = fully_connected(hidden2,num_outputs)

### Loss Function

In [76]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

### Optimizer

In [77]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

### Init

In [78]:
init = tf.global_variables_initializer()

In [81]:
training_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train,feed_dict={X:scaled_x_train,y_true:onehot_y_train })
        
    # Get Predictions
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits,axis=1)
    
    results = preds.eval()

In [82]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(results,y_test))

              precision    recall  f1-score   support

           0       1.00      0.35      0.52        54
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.35        54
   macro avg       0.33      0.12      0.17        54
weighted avg       1.00      0.35      0.52        54



D:\Install folder\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
